In [5]:
import time
import openai
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer
import signal

In [6]:
import random

In [7]:
# 创建一个函数，用于解决openai公司强制本程序暂停10分钟的问题（强制暂停每遍历500-2000条新闻发生一次）
def handler(signum, frame):
    raise Exception("Processing took too long")

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
encoding = 'utf-8' 
output= pd.read_csv('headline_组会.csv',encoding=encoding)

In [9]:
openai.api_key = ""

In [10]:
print(output)

         Date                                            content
0    20130612  When Words Just Aren't Enough, Some Turn to Fl...
1    20130613  NOW HEAR THIS: NAVY ABANDONS ALL CAPS --- Offi...
2    20130613  U.S. Oil Notches Record Growth --- Rise in Pro...
3    20130613  Rebels Plead for Weapons In Face of Syrian Ons...
4    20130613          Traders Pay for an Early Peek at Key Data
..        ...                                                ...
229  20130919             Analysis: Investors Now Left Wondering
230  20130919              Obama Weighs Talks With Iran at U.N .
231  20130919  Fed Stays the Course on Easy-Money --- Policy ...
232  20130919  'Three-Front War': Rebel-on-Rebel Violence Sei...
233  20130920                   Pricing Glitch in Health Rollout

[234 rows x 2 columns]


In [11]:
def process_row(index, output, df):
    row = output.loc[index]
    DATE = row['Date']
    news = row['content']
    messages = [
    {"role": "system", "content": ("Forget all previous instructions. You are now a financial expert giving investment advice.")},
    {"role": "user", "content":(" I'll give you a piece of news, "
    "and you need to determine whether the news is related to U.S. stock market. "
    "Please choose only one option from YES, NO, and do not provide any additional responses."
    "The news is: \n\n" + f"""{news}""")}
]

    # 调用OpenAI API生成文本*
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    
    reply = completion['choices'][0]['message']['content']
    row_data = {'news': news, 'reply': reply,'DATE': DATE}
    df = pd.concat([df, pd.DataFrame(row_data, index=[0])], ignore_index=True)
    time.sleep(0.5) #每遍历一则新闻，程序睡眠1秒
    return df

In [12]:
T1 = time.time()

In [13]:
# 创建一个变量来跟踪迭代的次数
counter = 0
index = 0  # 初始化索引
df = pd.DataFrame()
#len(output.index)
delay = 1  # 初始化延迟时间

# 设置信号处理程序
signal.signal(signal.SIGALRM, handler)

#len(output.index)
while index < len(output.index):
    try:
        
        # 设置一个60秒的计时器
        signal.alarm(40)
        
        df = process_row(index, output, df)  # 这里df获取了新的值
        index += 1  # 仅当成功处理了这一行时才增加索引
        
         # 关闭计时器
        signal.alarm(0)
        
        
        # 每90次迭代，就保存一个新的文件
        if counter % 50 == 0:
            df.to_csv('headline_组会_01.csv', index=False, encoding='utf-8-sig')
        
        # 每100次迭代，就输出一次当前的处理索引
        if counter % 10 == 0:
            print(f"处理到索引 {index}")
        
        # 不要忘记增加迭代计数器
        counter += 1
        delay = 1  # 重置延迟时间
    except Exception as e:
        print(f"处理索引 {index} 时发生错误: {e}")
        print(f"等待{delay}秒后再次尝试...")
        time.sleep(delay)  # 等待指定的秒数
        delay *= 2  # 指数级回退

处理到索引 1
处理到索引 11
处理到索引 21
处理到索引 31
处理到索引 41


KeyboardInterrupt: 

In [21]:
T2 = time.time()
print('程序运行时间:%s秒' % ((T2 - T1)))
print(df)

程序运行时间:162.21482968330383秒
                                                 news reply      DATE
0   When Words Just Aren't Enough, Some Turn to Fl...    NO  20130612
1   NOW HEAR THIS: NAVY ABANDONS ALL CAPS --- Offi...    NO  20130613
2   U.S. Oil Notches Record Growth --- Rise in Pro...   YES  20130613
3   Rebels Plead for Weapons In Face of Syrian Ons...    NO  20130613
4           Traders Pay for an Early Peek at Key Data   YES  20130613
..                                                ...   ...       ...
85  Fans Want Floppy the Dog to Sit and Stay, but ...    NO  20130713
86                     Snowden Seeks Asylum In Russia    NO  20130713
87  They Surely Do Love Their Name, Even If It Has...    NO  20130715
88           Feds Weigh Charging Zimmerman In Killing    NO  20130715
89  Pay-TV Providers Bid to End Sports Networks' W...   YES  20130715

[90 rows x 3 columns]


In [22]:
df.to_csv('headline_组会_01_存储.csv', index=False, encoding='utf-8-sig')